Connecting google colab
# Import libraries

In [ ]:
# Import libraries
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls "/content/gdrive/MyDrive/Colab/CSProto"

cwe_clean2.pkl


In [ ]:
# # Install necessary libraries
# !pip install tensorflow
# !pip install keras
# !pip install numpy
# !pip install pandas
# !pip install nltk
# !pip install scikit-learn

# the pickle file was setup with older version of pandas, so the older version is used
# !pip install pandas==1.4.2


In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, LSTM, Dense, Dropout, SpatialDropout1D, Bidirectional, GRU, Flatten
from keras.layers import TimeDistributed, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.utils import plot_model
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Embedding, LSTM, Bidirectional, TimeDistributed, concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
# Download necessary nltk tables

s

# Load data and drop unwanted columns

> CWE data 893 reports of data
>
> Source: https://cwe.mitre.org/


In [ ]:
# Current working directory(directory which contains these notebook)
# Unpickle the pickle file and Print x.head shows the head on the file content
#Change the directory to your file directory
#makae sure the file type is .pkl if not use oother method t import based on the dataset.
# import pickle

# with open('/content/gdrive/MyDrive/Colab/myfile.pkl', 'rb') as f:
#     cwe_data = pickle.load(f)

# print(cwe_data.head())


In [ ]:

# Current working directory(directory which contains these notebook)
# Unpickle the pickle file and Print x.head shows the head on the file content
import pickle

with open('/content/gdrive/MyDrive/Colab/CSProto/cwe_clean2.pkl', 'rb') as f:
    cwe_data = pickle.load(f)

print(cwe_data.head())


In [ ]:
import copy

# deep copy for the next phase dropping unnecessary data columns
cwe_deepcopy = copy.deepcopy(cwe_data)

# drop unwanted column
cwe_deepcopy.drop(['Authorization', 'Accountability', 'Authentication', 'Non_Repudiation', 'Availability', 'Confidentiality', 'Other'], axis=1, inplace=True)

# listing
list(cwe_deepcopy.columns.values)

In [ ]:
#created a new df ignoring multi tags and only selcting with a single tag
cwe_df = cwe_deepcopy
#see the shape
cwe_df.shape
#resetting the index
cwe_df.reset_index(drop = True)
cwe_df.head(5)

,id,name,description,extended_description,common_consequences_scope,common_consequences_impact,Clean_Description,Access_Control,Integrity
0,5,J2EE Misconfiguration: Data Transmission Witho...,Information sent over a network can be comprom...,None,"[Confidentiality, Integrity]","[Read Application Data, Modify Application Data]",information sent network compromised transit a...,0,1
1,6,J2EE Misconfiguration: Insufficient Session-ID...,The J2EE application is configured to use an i...,If an attacker can guess or steal a session ID...,[Access Control],[Gain Privileges or Assume Identity],j2ee application configured use insufficient s...,1,0
2,7,J2EE Misconfiguration: Missing Custom Error Page,The default error page of a web application sh...,\n,[Confidentiality],[Read Application Data],default error page web application display sen...,0,0
3,8,J2EE Misconfiguration: Entity Bean Declared Re...,When an application exposes a remote interface...,None,"[Confidentiality, Integrity]","[Read Application Data, Modify Application Data]",application exposes remote interface entity be...,0,1
4,9,J2EE Misconfiguration: Weak Access Permissions...,If elevated access rights are assigned to EJB ...,If the EJB deployment descriptor contains one ...,[Other],[Other],elevated access rights assigned ejb methods at...,0,0


In [ ]:
#count total index
len(cwe_df. index), cwe_df.columns

(895,
 Index(['id', 'name', 'description', 'extended_description',
        'common_consequences_scope', 'common_consequences_impact',
        'Clean_Description', 'Access_Control', 'Integrity'],
       dtype='object'))

In [ ]:
# converting list to string
cwe_df['common_consequences_scope'] = cwe_df['common_consequences_scope'].apply(''.join)

# get index to drop the index which has more than 1 label
dropIndex = cwe_df[(cwe_df['common_consequences_scope'] != "Access Control") & (cwe_df['common_consequences_scope'] != "Integrity")].index
# drop the row by index if has more than 1 label
cwe_df.drop(index=dropIndex, inplace=True)
cwe_df.head(2)][]

,id,name,description,extended_description,common_consequences_scope,common_consequences_impact,Clean_Description,Access_Control,Integrity
1,6,J2EE Misconfiguration: Insufficient Session-ID...,The J2EE application is configured to use an i...,If an attacker can guess or steal a session ID...,Access Control,[Gain Privileges or Assume Identity],j2ee application configured use insufficient s...,1,0
7,13,ASP.NET Misconfiguration: Password in Configur...,Storing a plaintext password in a configuratio...,None,Access Control,[Gain Privileges or Assume Identity],storing plaintext password configuration file ...,1,0


In [ ]:
# after dropping labels if contains more than one
cwe_df['common_consequences_scope'].unique(), cwe_df.shape[0]

(array(['Access Control', 'Integrity'], dtype=object), 179)

In [ ]:
cwe_df['Clean_Description'][0:3]

,Clean_Description
1,j2ee application configured use insufficient s...
7,storing plaintext password configuration file ...
31,software system accepts path input form traili...


In [ ]:
cwe_df_preprocess= cwe_df.copy()
cwe_df_preprocess.head(5)

,id,name,description,extended_description,common_consequences_scope,common_consequences_impact,Clean_Description,Access_Control,Integrity
1,6,J2EE Misconfiguration: Insufficient Session-ID...,The J2EE application is configured to use an i...,If an attacker can guess or steal a session ID...,Access Control,[Gain Privileges or Assume Identity],j2ee application configured use insufficient s...,1,0
7,13,ASP.NET Misconfiguration: Password in Configur...,Storing a plaintext password in a configuratio...,None,Access Control,[Gain Privileges or Assume Identity],storing plaintext password configuration file ...,1,0
31,42,Path Equivalence: 'filename.' (Trailing Dot),A software system that accepts path input in t...,None,Access Control,[Bypass Protection Mechanism],software system accepts path input form traili...,1,0
68,84,Improper Neutralization of Encoded URI Schemes...,The web application improperly neutralizes use...,None,Integrity,[Unexpected State],web application improperly neutralizes usercon...,0,1
76,93,Improper Neutralization of CRLF Sequences ('CR...,The software uses CRLF (carriage return line f...,None,Integrity,[Modify Application Data],software uses crlf carriage return line feeds ...,0,1


# Data Preprocessing and Tokenization

In [ ]:
import os
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token not in stop_words]
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

cwe_df_preprocess['Clean_Description'] = cwe_df_preprocess['Clean_Description'].apply(preprocess_text)


In [ ]:
MAX_FEATURES = 200000
MAX_SENTENCE_NUM = 40
MAX_WORD_NUM = 50
EMBED_SIZE = 100

sentences = cwe_df_preprocess['Clean_Description'].apply(lambda x: sent_tokenize(x)).tolist()

tokenizer = Tokenizer(num_words=MAX_FEATURES, lower=True, split=' ', oov_token='<OOV>')
tokenizer.fit_on_texts(cwe_df_preprocess['Clean_Description'])
sequences = tokenizer.texts_to_sequences(cwe_df_preprocess['Clean_Description'])

padded_sequences = pad_sequences(sequences, maxlen=MAX_WORD_NUM, padding='post', truncating='post')

word_index = tokenizer.word_index


In [ ]:
!ls "/content/gdrive/MyDrive/Colab/"

CSProto


In [ ]:
"""
A pre-trained word to vector is used from GloVe by Pennington et. al.
Source: https://nlp.stanford.edu/projects/glove/
The data was trained on wikipedia articles. Each word is described by a 100d vector.
"""

# Load word vectors from pre-trained dataset
embeddings_index = {}
f = open(os.path.join(os.getcwd(), '/content/gdrive/MyDrive/Colab/glove.6B.100d.txt'),encoding='UTF-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [ ]:
def create_embedding_matrix(word_index, embeddings, embedding_dim):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

embedding_matrix = create_embedding_matrix(word_index, embeddings_index, EMBED_SIZE)


In [ ]:
# Create embedding matrix
min_wordCount = 0
absent_words = 0
small_words = 0
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBED_SIZE))
word_counts = tokenizer.word_counts

for word, i in word_index.items():
    if word == '<OOV>':
        continue
    if word_counts[word] > min_wordCount:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            absent_words += 1
    else:
        small_words += 1

print('Total absent words are', absent_words, 'which is', "%0.2f" % (absent_words * 100 / len(word_index)),
      '% of total words')
print('Words with '+str(min_wordCount)+' or less mentions', small_words, 'which is', "%0.2f" % (small_words * 100 / len(word_index)),
      '% of total words')


Total absent words are 224 which is 36.90 % of total words
Words with 0 or less mentions 0 which is 0.00 % of total words


#Resetting index

In [ ]:
cwe_df_preprocess.head(5)

,id,name,description,extended_description,common_consequences_scope,common_consequences_impact,Clean_Description,Access_Control,Integrity
1,6,J2EE Misconfiguration: Insufficient Session-ID...,The J2EE application is configured to use an i...,If an attacker can guess or steal a session ID...,Access Control,[Gain Privileges or Assume Identity],j2ee applic configur use insuffici session id ...,1,0
7,13,ASP.NET Misconfiguration: Password in Configur...,Storing a plaintext password in a configuratio...,None,Access Control,[Gain Privileges or Assume Identity],store plaintext password configur file allow a...,1,0
31,42,Path Equivalence: 'filename.' (Trailing Dot),A software system that accepts path input in t...,None,Access Control,[Bypass Protection Mechanism],softwar system accept path input form trail do...,1,0
68,84,Improper Neutralization of Encoded URI Schemes...,The web application improperly neutralizes use...,None,Integrity,[Unexpected State],web applic improperli neutral usercontrol inpu...,0,1
76,93,Improper Neutralization of CRLF Sequences ('CR...,The software uses CRLF (carriage return line f...,None,Integrity,[Modify Application Data],softwar use crlf carriag return line feed spec...,0,1


In [ ]:
cwe_df_preprocess = cwe_df_preprocess.reset_index(drop=True)
cwe_df_preprocess.head(5)


,id,name,description,extended_description,common_consequences_scope,common_consequences_impact,Clean_Description,Access_Control,Integrity
0,6,J2EE Misconfiguration: Insufficient Session-ID...,The J2EE application is configured to use an i...,If an attacker can guess or steal a session ID...,Access Control,[Gain Privileges or Assume Identity],j2ee applic configur use insuffici session id ...,1,0
1,13,ASP.NET Misconfiguration: Password in Configur...,Storing a plaintext password in a configuratio...,None,Access Control,[Gain Privileges or Assume Identity],store plaintext password configur file allow a...,1,0
2,42,Path Equivalence: 'filename.' (Trailing Dot),A software system that accepts path input in t...,None,Access Control,[Bypass Protection Mechanism],softwar system accept path input form trail do...,1,0
3,84,Improper Neutralization of Encoded URI Schemes...,The web application improperly neutralizes use...,None,Integrity,[Unexpected State],web applic improperli neutral usercontrol inpu...,0,1
4,93,Improper Neutralization of CRLF Sequences ('CR...,The software uses CRLF (carriage return line f...,None,Integrity,[Modify Application Data],softwar use crlf carriag return line feed spec...,0,1


#Split data for trainng and testing

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Split the data into training and testing sets
#random_state=42 means that the random number generator used in the train_test_split function will be initialized with the seed value of 42.
# This ensures that the data split will be the same every time you run the code, making the results consistent and reproducible across different runs.
X = padded_sequences
y = cwe_df_preprocess[['Access_Control', 'Integrity']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Get the counts of the labels
val_counts = cwe_df_preprocess[['Access_Control', 'Integrity']].sum()

# Plot bar chart
plt.figure(figsize=(8, 6))
ax = sns.barplot(x=val_counts.index, y=val_counts.values, alpha=0.8)
plt.xlabel("Various Label Type")
plt.ylabel("Counts of the Labels")

rects = ax.patches
labels = val_counts.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2, height + 5, label, ha="center", va="bottom")

plt.show()


# HAN MODEL

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class AttentionLayer(Layer):
    def __init__(self, units, return_coefficients=False, **kwargs):
        self.units = units
        self.return_coefficients = return_coefficients
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='{}_W'.format(self.name))
        self.b = self.add_weight(shape=(input_shape[1], 1),
                                 initializer='zeros',
                                 trainable=True,
                                 name='{}_b'.format(self.name))
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        q = tf.nn.tanh(tf.tensordot(inputs, self.W, axes=1) + self.b)
        a = tf.nn.softmax(q, axis=1)
        outputs = tf.reduce_sum(inputs * a, axis=1)

        if self.return_coefficients:
            return outputs, a
        return outputs

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[1])]
        return input_shape[0], input_shape[-1]


In [ ]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


X_train shape: (143, 50)
X_test shape: (36, 50)


In [ ]:
tf.random.set_seed(24)
# Define the embedding layer
embedding_layer = Embedding(len(word_index) + 1, EMBED_SIZE, weights=[embedding_matrix],
                            input_length=MAX_WORD_NUM, trainable=False, name='word_embedding')

# Words level attention model
word_input = Input(shape=(MAX_WORD_NUM,), dtype='int32', name='word_input')
word_sequences = embedding_layer(word_input)
word_gru = Bidirectional(GRU(48, return_sequences=True), name='word_gru')(word_sequences)
word_dense = Dense(100, activation='relu', name='word_dense')(word_gru)
word_att, word_coeffs = AttentionLayer(EMBED_SIZE, True, name='word_attention')(word_dense)

# Adding a dropout layer
word_drop = Dropout(0.6, name='word_dropout')(word_att)

# Adding the output layer
preds = Dense(2, activation='softmax', name='output')(word_drop)

# Model compile
model = Model(word_input, preds)
optimizer = Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])
print(model.summary())


## Train

In [ ]:
# Train the model
history = model.fit(X_train, y_train, batch_size=10, epochs=20, validation_split=0.2)



# Visualize the model
plot_model(model, to_file='model.png', show_shapes=True)

The training seems to be overfitting the data since the training accuracy is much higher than the validation accuracy. The validation accuracy does not increase beyond 88.89% which indicates that the model has reached its limit of learning from the given data. This could be because of a limited dataset or the model architecture is not capable of capturing more information from the data. It is a good idea to try different hyperparameters and architectures to see if the model's performance can be improved. Additionally, we can also try data augmentation techniques to increase the amount of data and improve the model's performance.

#Trainning

In [ ]:
loss, acc = model.evaluate(X_train, y_train)
print('Trainning loss:', loss)
print('Traainning accuracy:', acc)

5/5 [==============================] - 0s 21ms/step - loss: 0.3553 - acc: 0.8811
Trainning loss: 0.3553365170955658
Traainning accuracy: 0.881118893623352


In [ ]:
print(history.history.keys())
# Plot of accuracy in each epoch
plt.figure(figsize=(10,8))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# Plot of loss in each epoch
plt.figure(figsize=(10,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Test

In [ ]:
# Evaluate the model on the test set
loss, acc = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', acc)

2/2 [==============================] - 0s 18ms/step - loss: 0.3903 - acc: 0.8333
Test loss: 0.3903241753578186
Test accuracy: 0.8333333134651184


## BilStM Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

# Define model architecture
model_2 = Sequential()
model_2.add(Embedding(input_dim=num_words, output_dim=EMBED_SIZE, input_length=MAX_WORD_NUM, weights=[embedding_matrix], trainable=False))
model_2.add(Bidirectional(LSTM(units=128, return_sequences=True, dropout=0.4, recurrent_dropout=0.4)))
model_2.add(Bidirectional(LSTM(units=64, dropout=0.2, recurrent_dropout=0.4)))
model_2.add(Dense(units=2, activation='softmax'))

# Compile model
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# Print model summary
model_2.summary()



In [ ]:
# Train model
history2 = model_2.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=10)




In [ ]:
# Plot training and validation accuracy
plt.plot(history2.history['acc'])
plt.plot(history2.history['val_acc'])
plt.title('Model 2 Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training and validation loss
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('Model 2 Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
loss, acc = model.evaluate(X_train, y_train)
print('Test loss:', loss)
print('Test accuracy:', acc)

5/5 [==============================] - 0s 19ms/step - loss: 0.3553 - acc: 0.8811
Test loss: 0.3553365170955658
Test accuracy: 0.881118893623352


In [ ]:
# Evaluate the model on the test set
loss, acc = model_2.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', acc)

2/2 [==============================] - 0s 41ms/step - loss: 0.8018 - acc: 0.8056
Test loss: 0.8017815351486206
Test accuracy: 0.8055555820465088


**TODO: The test accuracy is too low, make it high by using other Models rather than the HAN classifier.**